In [9]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3
load_dotenv()
from keylog_analysis_helper import *

Notes
- Code reconstruction may be incorrect, double check again
- Measures of progress:
    - Whether code compiles,
    - Ask llm if it progressed?
    - '# characters written
        - Those that are code and not (ask llm)


- does this portion make sense
- does this portion make sense to the solution 


Define model

In [8]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="models/gemini-1.5-pro-002",
  # generation_config=generation_config,
)

# chat_session = model.start_chat(
#   history=[
#   ]
# )

Define Loop

In [ ]:
a3 = True

df = pd.read_csv("KeylogDatasets/a3.csv") if a3 else pd.read_csv("KeylogDatasets/a4.csv")
p_list = p_list_a3_log_analysis if a3 else p_list_a4_log_analysis
assignment = 'a3' if a3 else 'a4'
snapshots_dir = snapshots_dir_a3 if a3 else snapshots_dir_a4

for p in p_list:
    df_temp = df[df["participant"] == p]

    # Create chat session
    chat_session = model.start_chat(
        history=[
            row["text"]
        ]
    )

    for idx, row in df_temp.iterrows()[1:]:
        print(row["participant"], row['snapshot'])

        # read snapshot file path
        # Read file 1 if change was made
        prev_snapshot_filename = f'{assignment}_{p}_c{row['snapshot']-1}'
        prev_snapshot_path = os.path.join(snapshots_dir, prev_snapshot_filename)
        prev_snapshot = read_file_as_string(prev_snapshot_path)
        current_snapshot_filename = f'{assignment}_{p}_c{row['snapshot']}'
        current_snapshot_path = os.path.join(snapshots_dir, current_snapshot_filename)
        current_snapshot_path = read_file_as_string(current_snapshot_path)

        old_file_1 = None
        current_file_1 = None

        # Read file 2 if change was made
        old_file_2 = None
        current_file_2 = None

        # Read autograder code
        autograder_2 = None

        # Ask the llm
        prompt = None

        response = chat_session.send_message()
        print(response)
        df.at[idx, "llm_response"] = response
        df.to_csv("KeylogDatasets/a3_llm.csv", index=False) if a3 else df.to_csv("KeylogDatasets/a4.csv", index=False)
        time.sleep(1)